In [1]:
from scipy.io import loadmat
import numpy as np

# Load the .mat file
data = loadmat(r'C:\Users\vishw\Documents\karafeature\MM08.mat')
lab =loadmat(r'C:\Users\vishw\Documents\karafeature\LabelMM081.mat')
# Extract features and labels
features = data['all_features']
labels = lab['all_labels'].flatten()
# labels = lab['all_labels'].flatten()  # Flatten if labels are in (8184, 1) shape
# If labels start from 1, adjust them to start from 0
if np.min(labels) == 1:
    labels -= 1

In [2]:
import numpy as np
import time
from sklearn.preprocessing import StandardScaler

# Optimized activation functions using NumPy
def selu(x):
    alpha = 1.6732632423543772
    scale = 1.0507009873554805
    return scale * np.where(x > 0, x, alpha * (np.exp(x) - 1))

def leaky_relu(x):
    return np.where(x > 0, x, 0.1 * x)

def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def hardlim(x):
    return np.where(x > 0, 1, 0)

def tribas(x):
    return np.maximum(1 - np.abs(x), 0)

def radbas(x):
    return np.exp(-x**2)

# Mapping activation functions to their respective functions
activation_functions = {
    1: selu,
    2: relu,
    3: sigmoid,
    4: np.sin,
    5: hardlim,
    6: tribas,
    7: radbas,
    8: np.sign,
    9: leaky_relu,
    10: np.tanh
}

# One-hot encoding function
def one_hot(x, n_class):
    y = np.zeros((len(x), n_class))
    for i in range(n_class):
        y[x == i, i] = 1
    return y

# Evaluation metrics
def Evaluate(ACTUAL, PREDICTED):
    p = np.sum(ACTUAL == 1)
    n = np.sum(ACTUAL == 0)
    
    tp = np.sum((ACTUAL == 1) & (PREDICTED == 1))
    tn = np.sum((ACTUAL == 0) & (PREDICTED == 0))
    fp = np.sum((ACTUAL == 0) & (PREDICTED == 1))
    fn = np.sum((ACTUAL == 1) & (PREDICTED == 0))
    
    accuracy = 100 * (tp + tn) / (p + n)
    sensitivity = 100 * tp / p if p != 0 else 0
    specificity = 100 * tn / n if n != 0 else 0
    precision = 100 * tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = sensitivity
    f_measure = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0
    gmean = 100 * np.sqrt((tp / p) * (tn / n)) if p != 0 and n != 0 else 0
    
    return [accuracy, sensitivity, specificity, precision, recall, f_measure, gmean, tp, tn, fp, fn]

def RVFL_train(train_data, test_data, C, N, activation):
    start = time.time()
    
    # Split training data
    trainX = train_data[:, :-1]
    trainY = train_data[:, -1]
    
    # One-hot encode labels
    nclass = 2
    trainY_encoded = one_hot(trainY.astype(int), nclass)
    
    # Initialize random weights and biases
    Nsample, Nfea = trainX.shape
    W = np.random.uniform(-1, 1, (Nfea, N))
    b = 0.1 * np.random.rand(1, N)
    
    # Compute hidden layer output for training data
    X1 = np.dot(trainX, W) + b
    activation_func = activation_functions.get(activation, relu)
    X1 = activation_func(X1)
    
    # Augment the input with hidden layer output and bias term
    X = np.hstack((trainX, X1, np.ones((Nsample, 1))))
    
    # Compute output weights (beta) using regularized least squares
    if X.shape[1] < Nsample:
        beta = np.linalg.inv(np.eye(X.shape[1]) / C + X.T @ X) @ X.T @ trainY_encoded
    else:
        beta = X.T @ np.linalg.inv(np.eye(X.shape[0]) / C + X @ X.T) @ trainY_encoded

    # Test phase
    testX = test_data[:, :-1]
    testY = test_data[:, -1]
    
    X1_test = np.dot(testX, W) + b
    X1_test = activation_func(X1_test)
    
    # Augment test data
    X_test = np.hstack((testX, X1_test, np.ones((testX.shape[0], 1))))
    
    # Calculate scores and classify
    raw_scores = X_test @ beta
    predicted_labels = np.argmax(raw_scores, axis=1)
    
    # Evaluate performance
    eval_metrics = Evaluate(testY, predicted_labels)
    end = time.time()
    
    return eval_metrics, end - start


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


dataset = np.hstack((features, labels.reshape(-1, 1)))

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42, stratify=labels)

scaler = StandardScaler()
train_data[:, :-1] = scaler.fit_transform(train_data[:, :-1])
test_data[:, :-1] = scaler.transform(test_data[:, :-1])

C = 0.01       # Regularization parameter
N = 100        # Number of random hidden neurons
activation = 2  # Activation function index (e.g., 2 for ReLU)

eval_metrics, duration = RVFL_train(train_data, test_data, C, N, activation)

print("Evaluation Metrics (Accuracy, Sensitivity, Specificity, Precision, Recall, F1 Score, G-Mean):")
print(eval_metrics)
print("Training and Testing Time:", duration, "seconds")


Evaluation Metrics (Accuracy, Sensitivity, Specificity, Precision, Recall, F1 Score, G-Mean):
[97.54385964912281, 96.32352941176471, 98.65771812080537, 98.49624060150376, 96.32352941176471, 97.39776951672864, 97.48363766862103, 131, 147, 2, 5]
Training and Testing Time: 4.319603681564331 seconds
